In [216]:
import bs4
import lxml
import pandas as pd
import urllib
import csv
import re

from urllib import request

In [217]:
base_url = "https://www.century21.fr/annonces/location-appartement/v-paris/"
url = "https://www.century21.fr/annonces/location-appartement/v-paris/page-2/"
request_text = request.urlopen(base_url).read()
page = bs4.BeautifulSoup(request_text, "lxml")

In [218]:
# Fonction pour remplacer la première virgule par un point avant "m2"
def replace_comma_with_dot_before_m2(text):
    return re.sub(r'(\d+),(\d+\s*m2)', r'\1.\2', text, 1)

def delete_m2(text):
    return re.sub(r'(\d+\s*) m2', r'\1', text, 1)

def delete_pieces(text):
    return re.sub(r' (\d+\s*) pièce.*', r'\1', text, 1)

def delete_euro(text):
    return re.sub(r'€', '', text, 1)


def get_info(annonces):
    infos = []
    for annonce in annonces:
        info = annonce.find(class_="c-text-theme-heading-4 tw-text-c21-grey-darker tw-font-semibold")
        prix = annonce.find(class_="c-text-theme-heading-1 is-constant-size-on-mobile tw-mt-2 tablet-landscape:tw-mt-0 tw-whitespace-nowrap")
        line = info.get_text().strip().split("\n")
        res = []
        res.append(line[0].strip())
        res.append(line[2].strip())
        modified_string = line[5].strip()
        modified_string = replace_comma_with_dot_before_m2(modified_string).split(",")
        modified_string[0] = delete_m2(modified_string[0])
        modified_string[1] = delete_pieces(modified_string[1])
        res.append(modified_string[0])
        res.append(modified_string[1])
        modified_string = prix.get_text().strip().split("\n")[0]
        modified_string = modified_string.replace(" ", "")
        modified_string = delete_euro(modified_string)
        res.append(modified_string)
        infos.append(res)
    return infos


annonces = page.find_all(class_="js-the-list-of-properties-list-property")
data = get_info(annonces)
while True:
    a_tags = page.find_all(class_="c-the-pagination-bar__item tw-flex tw-items-center tw-justify-center tw-w-10 tw-h-10 tw-bg-c21-gold tw-text-white tw-transition-colors tw-duration-200")
    print(len(a_tags))
    if len(a_tags) == 1 and a_tags[0].get('aria-label') == "prev":
        break

    for a_tag in a_tags:
        if a_tag.get('aria-label') == "next":
            url = "https://www.century21.fr" + a_tag.get('href')
            request_text = request.urlopen(url).read()
            page = bs4.BeautifulSoup(request_text, "lxml")

    annonces = page.find_all(class_="js-the-list-of-properties-list-property")
    data += get_info(annonces)

1
2
2
1


In [219]:
filename = 'century21Paris.csv'
labels = ['Ville', 'Arrondissement', 'Surface (m2)', 'Pièce(s)', 'Prix (€)']
data.insert(0, labels)
# Création et écriture dans le fichier CSV
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Écriture de chaque ligne sélectionnée dans le fichier CSV
    for line in data:
        writer.writerow([line[0], line[1], line[2], line[3], line[4]])

print(f'Fichier "{filename}" créé avec succès.')

Fichier "century21Paris.csv" créé avec succès.
